In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import flopy.plot.styles as styles
from dis2disu import Dis2Disu

In [ ]:
Lx = 5.0
Lz = 3.0
nlay, nrow, ncol = 2, 1, 10
delc = 1.
delr = float(Lx / ncol)
delz = float(Lz / nlay)
top = np.linspace(.5 * delr, Lx - .5 * delr, ncol)
top = top.reshape((nrow, ncol))
botm = np.empty((nlay, nrow, ncol))
botm[0] = top - delz
for k in range(1, nlay):
    botm[k] = botm[k - 1] - delz

delr = delr * np.ones(ncol, dtype=float)
delc = delc * np.ones(nrow, dtype=float)
mgs = flopy.discretization.StructuredGrid(delr=delr, delc=delc, top=top, botm=botm)
xs = flopy.plot.PlotCrossSection(modelgrid=mgs, line={"row":0})
xs.plot_grid()

d2d = Dis2Disu(delr, delc, top, botm, staggered=True)

In [ ]:
ws = './disu_model'
name = 'mymodel'
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name='mf6')
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim, linear_acceleration='bicgstab')
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
disu = flopy.mf6.ModflowGwfdisu(gwf, **d2d.get_gridprops_disu6())
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, xt3doptions=True,
                              angle1=0., angle2=0., angle3=0.)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0,), 1.], [(10,), 1.], [(9,), 0.], [(19,), 0.]])
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord=budget_file,
                            head_filerecord=head_file,
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])
sim.write_simulation()
sim.run_simulation()

head = gwf.output.head().get_data()
bud = gwf.output.budget()
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0.0, 0.5), (5.0, 0.5)]})
xs.plot_array(head)
xs.plot_grid()
xs.plot_vector(qx, qy, qz, normalize=True, color="white")